In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.externals import joblib
import cv2
import time
import numpy as np
import pickle
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from matplotlib import pyplot as plt

## classifiers
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

plt.rc('text', usetex=True)
plt.rc('font', family='serif')



# names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
#          "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
#          "Naive Bayes", "QDA"]

# classifiers = [
#     KNeighborsClassifier(3),
#     SVC(kernel="linear", C=0.025),
#     SVC(gamma=2, C=1),
#     GaussianProcessClassifier(1.0 * RBF(1.0), warm_start=True),
#     DecisionTreeClassifier(max_depth=5),
#     RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
#     MLPClassifier(alpha=1),
#     AdaBoostClassifier(),
#     GaussianNB(),
#     QuadraticDiscriminantAnalysis()]

feature_norms_c = [[0.066107951, 0.093825646], [0.076604679, 0.1844517], [0.14459303, 0.25104082], [1.5193609e-06, 2.120856e-05], [9.1782988e-07, 2.9007275e-05], [1.967471e-06, 7.1517374e-05], [2.6846066e-05, 0.0010604195], [8.5266156e-06, 5.3341791e-05], [1.129209e-05, 7.1516573e-05], [0.48276466, 0.43917835], [0.0093993777, 0.053004254], [0.011718377, 0.044239696], [3.4864228e-05, 0.00012562591], [3.5853725e-05, 0.00013510714], [0.0038112549, 0.03273331], [0.0022042503, 0.017467843], [0.017346229, 0.10239599], [4.6682849e-06, 3.6901329e-05], [6.4126789e-06, 4.4451961e-05], [2.9053201e-06, 2.3045086e-05], [0.0012592325, 0.017966012], [1.8832974e-06, 1.7578202e-05], [5.0102985e-06, 6.9388007e-05], [4.8721697e-05, 0.00049804652], [0.03282835, 0.11372703], [0.31020662, 0.39395973], [1.2811007e-05, 0.00055486365], [0.0032934344, 0.035110608], [0.00011244637, 0.0011812642], [0.0011822763, 0.018472912], [0.00066970702, 0.014979535], [0.00024093642, 0.005531359], [0.097702958, 0.24222577], [0.012759583, 0.068540044], [0.00011811391, 0.0025238025], [6.5307293e-05, 0.0015006483], [0.012112634, 0.089681715]]
feature_norms_v = [[1.5193609e-06, 2.120856e-05], [9.1782988e-07, 2.9007275e-05], [1.967471e-06, 7.1517374e-05], [2.6846066e-05, 0.0010604195], [8.5266156e-06, 5.3341791e-05], [1.129209e-05, 7.1516573e-05], [0.48276466, 0.43917835], [0.0093993777, 0.053004254], [0.011718377, 0.044239696], [3.4864228e-05, 0.00012562591], [3.5853725e-05, 0.00013510714], [0.0038112549, 0.03273331], [0.0022042503, 0.017467843], [0.017346229, 0.10239599], [4.6682849e-06, 3.6901329e-05], [6.4126789e-06, 4.4451961e-05], [2.9053201e-06, 2.3045086e-05], [0.0012592325, 0.017966012], [1.8832974e-06, 1.7578202e-05], [5.0102985e-06, 6.9388007e-05], [4.8721697e-05, 0.00049804652], [0.03282835, 0.11372703], [0.31020662, 0.39395973], [1.2811007e-05, 0.00055486365], [0.0032934344, 0.035110608], [0.00011244637, 0.0011812642], [0.0011822763, 0.018472912], [0.00066970702, 0.014979535], [0.00024093642, 0.005531359], [0.097702958, 0.24222577], [0.012759583, 0.068540044], [0.00011811391, 0.0025238025], [6.5307293e-05, 0.0015006483], [0.012112634, 0.089681715]]
feature_norms_g = [[0.066107951, 0.093825646], [0.076604679, 0.1844517], [0.14459303, 0.25104082]]


In [2]:

def normolize_data(data, feature_norms):
    for i in range(len(data[0])):
        feature = np.zeros( [len(data), 1], dtype=np.float32 )
        mean = feature_norms[i][0]
        std = feature_norms[i][1]
        for j in range(len(data)):
            data[j][i] = (data[j][i] - mean)/std

    return data

def normolize_dataset(x):
    feature_norms = []
    for i in range(len(x[0])):
        feature = np.zeros( [len(x), 1], dtype=np.float32 )
        for j in range(len(x)):
            feature[j] = x[j][i]

        norm = []
        norm.append(np.mean(feature))
        norm.append(np.std(feature))
        feature_norms.append(norm)
    
    return feature_norms


def voted_prediction(img, row, col, radius, ori_predict):
    start_row = row - radius/2
    end_row   = row + radius/2
    start_col = col - radius/2
    end_col   = col + radius/2

    img_rows, img_cols = img.shape

    if start_row < 0:
        start_row = 0
    if start_col < 0:
        start_col = 0

    if end_row > img_rows:
        end_row = img_rows
    if end_col > img_cols:
        end_col = img_cols        

    sub_img = img[start_row:end_row, start_col:end_col]
    hist,bins = np.histogram(sub_img.ravel(),6,[0,256])
    hist = hist[1:]
    new_predit = hist.argmax()+1

    return new_predit


def get_feature(rosbag_name, type = 'c', index_start = 2, index_end = 12):
    base_path_geo_result = '/home/xi/workspace/labels/output/' + rosbag_name + '/' + rosbag_name + '_'
    pickle_path = '/home/xi/workspace/labels/' + rosbag_name + '/' + rosbag_name + '_'
    total_sample = 0
    safe_sample = 0
    risky_sample = 0
    obstacle_sample = 0

    for i in range(index_start, index_end):   
        pick = pickle.load( open( pickle_path + str(i) + '.p', "rb" ) )
        # print pick.shape
        file_path = base_path_geo_result + str(i) + '_features.txt'
        print file_path
        with open(file_path) as f:
            content = f.readlines()
#             img_lengths.append(img_lengths[-1] + len(content))
            for line in content:
                # print line
                features = line.split(' ')
                
                features_clean = []
                for feature in features:
                    feature = feature.replace('\n', '')
                    features_clean.append(float(feature))
                # print features_clean

                if features_clean[0] < 1 or features_clean[0] > 3:
#                     img_lengths[-1] -= 1 
                    continue

                # if features_clean[0] == 1 or features_clean[0] == 2:
                #     features_clean[0] = 1

                h_d = features_clean[2]
                s   = features_clean[3]
                r   = features_clean[4]
                d   = features_clean[5]

                row = int(features_clean[7]*255/540)
                col = int(features_clean[8]*512/960)

            
                # if features_clean[2] > 0.3:
                #     features_clean[0] = 3

                x_new = []

                feature_v = []
                for v_index in xrange(34):
                    f_v = pick[-1][row][col][v_index]
                    # print f_v
                    feature_v.append(f_v)

                if type == 'g':
                    x_new = features_clean[2:5]
                elif type == 'v':
                    # x_new.append(features_clean[6]) # vision
                    x_new.extend(feature_v)
                elif type == 'c':
                    x_new = features_clean[2:5]
                    x_new.extend(feature_v)
                    # x_new.append(features_clean[6]) # vision

                for k in xrange(len(x_new)):
                    if type == 'g':
                        mean = feature_norms_g[k][0]
                        std = feature_norms_g[k][1]
                    if type == 'v':
                        mean = feature_norms_v[k][0]
                        std = feature_norms_v[k][1]                    
                    if type == 'c':
                        mean = feature_norms_c[k][0]
                        std = feature_norms_c[k][1]   
                    x_new[k] = (x_new[k] - mean)/std  
                        
                y_new = int(features_clean[0]) # true label

                x.append(x_new)
                y.append(y_new)

                uv = features_clean
#                 img_uvs.append(uv)

                total_sample += 1
                if y_new == 1:
                    safe_sample += 1
                elif y_new == 2:
                    risky_sample += 1
                elif y_new == 3:
                    obstacle_sample += 1
                # img_uvs.append(i)

    img_uvs = []
    img_lengths = []
    # print total_sample, safe_sample, risky_sample, obstacle_sample
    return x, y, img_lengths, img_uvs

 
def get_result(x_train, x_test, y_train, y_test):
    y_test = label_binarize(y_test, classes=[1,2,3])
    y_train= label_binarize(y_train, classes=[1,2,3])
    n_classes = y_test.shape[1]

    for name, clf in zip(names, classifiers):

        print name
        # clf.fit(x_train, y_train)
        # # scores = clf.score(x_test, y_test)
        # predict = clf.predict(x_test)
        # print metrics.confusion_matrix(y_test, predict)
        # print metrics.classification_report(y_test, predict)

        classifier = OneVsRestClassifier(clf)
        classifier.fit(x_train, y_train)
        predict = classifier.predict(x_test)
        y_score = classifier.predict_proba(x_test)
        y_labels = classifier.predict(x_test)

        print metrics.classification_report(y_test, predict)
        # # print metrics.confusion_matrix(y_test, y_labels)
        # # print y_score, y_labels


        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for i in range(3):
            fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])

        # Compute micro-average ROC curve and ROC area    
        fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    return fpr, tpr, roc_auc


def draw_single_ROC(fpr_c, tpr_c, roc_auc_c, fpr_v, tpr_v, roc_auc_v, fpr_g, tpr_g, roc_auc_g, type='safe'):
    index = 0
    if type == 'risky':
        index = 1
    elif type == 'obstacle':
        index = 2
    plt.figure(figsize=(5, 5))
    plt.plot(fpr_c[index], tpr_c[index],
            label=r'Fusion ({0:0.2f}\% AUC)'.format(100 *roc_auc_c[index]),
            color='deeppink', linestyle='-', linewidth=2)            
    plt.plot(fpr_g[index], tpr_g[index],
            label=r'Geometry ({0:0.2f}\% AUC)'.format(100 * roc_auc_g[index]),
            color='cornflowerblue', linestyle='-', linewidth=2)
    plt.plot(fpr_v[index], tpr_v[index],
            label=r'Vision ({0:0.2f}\% AUC)'.format(100 * roc_auc_v[index]),
            color='darkorange', linestyle='-', linewidth=2)    
    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel(r'False Positive Rate', fontsize=14)
    plt.ylabel(r'True Positive Rate', fontsize=14)
    # plt.title('Classification result on ' + type + ' terrain')
    plt.legend(loc="lower right")
    plt.savefig('roc_' + type, bbox_inches='tight')
    plt.show()



def draw_ROC(fpr_c, tpr_c, roc_auc_c, fpr_v, tpr_v, roc_auc_v, fpr_g, tpr_g, roc_auc_g):
    # flat terrain
    draw_single_ROC(fpr_c, tpr_c, roc_auc_c, fpr_v, tpr_v, roc_auc_v, fpr_g, tpr_g, roc_auc_g, 'safe')
    draw_single_ROC(fpr_c, tpr_c, roc_auc_c, fpr_v, tpr_v, roc_auc_v, fpr_g, tpr_g, roc_auc_g, 'risky')
    draw_single_ROC(fpr_c, tpr_c, roc_auc_c, fpr_v, tpr_v, roc_auc_v, fpr_g, tpr_g, roc_auc_g, 'osbtacle')



def draw_result_img(x_test, y_test, x_train, y_train, img_lengths_test, img_uvs_test, type):
    # y_vote = predict[:]
    clf = RandomForestClassifier(max_depth=5, n_estimators=15)
    # classifier = OneVsRestClassifier(clf)
    clf.fit(x_train, y_train)
    predict = clf.predict(x_test)

    for img_index in range(len(img_lengths_test)-1):
        print img_index, len(img_lengths_test)-1
        #############################################################################
        ### draw result for one image
        true_img = np.zeros([540,960, 3], dtype=np.uint8)
        predict_img = np.zeros([540,960, 3], dtype=np.uint8)
        new_predit_img = np.zeros([540,960], dtype=np.uint8)

        x_one_img = x_test[img_lengths_test[img_index]:img_lengths_test[img_index+1]]
        y_one_img = y_test[img_lengths_test[img_index]:img_lengths_test[img_index+1]]
        uv_one_img = img_uvs_test[img_lengths_test[img_index]:img_lengths_test[img_index+1]]

        predict_one_img = clf.predict(x_one_img)
        # print 'before voting'
        # print metrics.confusion_matrix(y_one_img, predict_one_img)
        # print metrics.classification_report(y_one_img, predict_one_img)

        for i in range(len(x_one_img)):
            row = int(uv_one_img[i][-2])
            col = int(uv_one_img[i][-1]) 

            radius = 20.0/540.0 * row 

            # cv2.circle(true_img, (col, row), int(radius), y_one_img[i] * 50, -1)
            if y_one_img[i] == 0:
                cv2.circle(true_img, (col, row), int(radius), [0, 0, 200], -1)
            if y_one_img[i] == 1:
                cv2.circle(true_img, (col, row), int(radius), [0, 200, 0], -1)
            if y_one_img[i] == 2:
                cv2.circle(true_img, (col, row), int(radius), [200, 0, 0], -1)

            # cv2.circle(predict_img, (col, row), int(radius), predict_one_img[i] * 50, -1)
            # true_img[row,col] = y_one_img[i] * 50
            # predict_img[row,col] = predict_one_img[i] * 50
            # window_size = 30 + 100/540 * row
            # new_predit = voted_prediction(predict_img, row, col, 50, predict_one_img[i])
            # cv2.circle(predict_img, (col, row), int(radius), predict_one_img[i] * 50, -1)
            if predict_one_img[i] == 1:
                cv2.circle(predict_img, (col, row), int(radius), [0, 255, 0], -1)
            if predict_one_img[i] == 2:
                cv2.circle(predict_img, (col, row), int(radius), [0, 255, 255], -1)
            if predict_one_img[i] == 3:
                cv2.circle(predict_img, (col, row), int(radius), [0, 0, 255], -1)

            # predict_one_img[i] = new_predit
        
        # y_vote[img_lengths_test[img_index]:img_lengths_test[img_index+1]] = predict_one_img

        path = type + str(img_index) + '.png'
        path_t = type + str(img_index) + '_t.png'
        # cv2.imshow("true", true_img)
        # cv2.imshow("predict_img", predict_img)
        cv2.imwrite(path, predict_img)
        # cv2.imwrite(path_t, true_img)
        # cv2.imshow("voted_img", new_predit_img)
        # cv2.waitKey(0)
    ###########################################################################

def load_data(data_list, feature_type):
    for index in data_list:
        index = index - 1
        x_train_c, y_train_c, img_lengths_train_c, img_uvs_train_c = get_feature(data_names[index], x_train_g, y_train_g, img_lengths_train_g, img_uvs_train_g, feature_type)

    return x_train_c, y_train_c, img_lengths_train_c, img_uvs_train_c


In [3]:
x = []
y = []
x_train_c = []
y_train_c = []
x_test_c = []
y_test_c = []

x_train_v = []
y_train_v = []
x_test_v = []
y_test_v = []

x_train_g = []
y_train_g = []
x_test_g = []
y_test_g = []
################################ get trainning and test set ############################

img_lengths_train_c = []
img_uvs_train_c = []
img_lengths_train_g = []
img_uvs_train_g = []
img_lengths_train_v = []
img_uvs_train_v = []

img_lengths_train_c.append(0)
img_lengths_train_g.append(0)
img_lengths_train_v.append(0)

# x_train, y_train, img_lengths_train, img_uvs_train = get_feature('hogwarts', x_train, y_train, img_lengths_train, img_uvs_train)

data_names = ['hogwarts', 'snow_grass', 'slope', 'parking', 'trash_summer', 'trash_winter', 'stain']

## testing set
img_lengths_test_c = []
img_uvs_test_c = []
img_lengths_test_g = []
img_uvs_test_g = []
img_lengths_test_v = []
img_uvs_test_v = []
img_lengths_test_c.append(0)
img_lengths_test_g.append(0)
img_lengths_test_v.append(0)



In [4]:
index = 0
x_1_c, y_1_c, img_lengths_1_c, img_uvs_1_c = get_feature(data_names[index], 'c')
x_1_v, y_1_v, img_lengths_1_v, img_uvs_1_v = get_feature(data_names[index], 'v')
x_1_g, y_1_g, img_lengths_1_g, img_uvs_1_g = get_feature(data_names[index], 'g')

index = 1
x_2_c, y_2_c, img_lengths_1_c, img_uvs_1_c = get_feature(data_names[index], 'c')
x_2_v, y_2_v, img_lengths_1_v, img_uvs_1_v = get_feature(data_names[index], 'v')
x_2_g, y_2_g, img_lengths_1_g, img_uvs_1_g = get_feature(data_names[index], 'g')

index = 2
x_3_c, y_3_c, img_lengths_1_c, img_uvs_1_c = get_feature(data_names[index], 'c')
x_3_v, y_3_v, img_lengths_1_v, img_uvs_1_v = get_feature(data_names[index], 'v')
x_3_g, y_3_g, img_lengths_1_g, img_uvs_1_g = get_feature(data_names[index], 'g')

index = 3
x_4_c, y_4_c, img_lengths_1_c, img_uvs_1_c = get_feature(data_names[index], 'c')
x_4_v, y_4_v, img_lengths_1_v, img_uvs_1_v = get_feature(data_names[index], 'v')
x_4_g, y_4_g, img_lengths_1_g, img_uvs_1_g = get_feature(data_names[index], 'g')

index = 4
x_5_c, y_5_c, img_lengths_1_c, img_uvs_1_c = get_feature(data_names[index], 'c')
x_5_v, y_5_v, img_lengths_1_v, img_uvs_1_v = get_feature(data_names[index], 'v')
x_5_g, y_5_g, img_lengths_1_g, img_uvs_1_g = get_feature(data_names[index], 'g')

index = 5
x_6_c, y_6_c, img_lengths_1_c, img_uvs_1_c = get_feature(data_names[index], 'c')
x_6_v, y_6_v, img_lengths_1_v, img_uvs_1_v = get_feature(data_names[index], 'v')
x_6_g, y_6_g, img_lengths_1_g, img_uvs_1_g = get_feature(data_names[index], 'g')

index = 6
x_7_c, y_7_c, img_lengths_1_c, img_uvs_1_c = get_feature(data_names[index], 'c')
x_7_v, y_7_v, img_lengths_1_v, img_uvs_1_v = get_feature(data_names[index], 'v')
x_7_g, y_7_g, img_lengths_1_g, img_uvs_1_g = get_feature(data_names[index], 'g')



/home/xi/workspace/labels/output/hogwarts/hogwarts_2_features.txt
/home/xi/workspace/labels/output/hogwarts/hogwarts_3_features.txt
/home/xi/workspace/labels/output/hogwarts/hogwarts_4_features.txt
/home/xi/workspace/labels/output/hogwarts/hogwarts_5_features.txt
/home/xi/workspace/labels/output/hogwarts/hogwarts_6_features.txt
/home/xi/workspace/labels/output/hogwarts/hogwarts_7_features.txt
/home/xi/workspace/labels/output/hogwarts/hogwarts_8_features.txt
/home/xi/workspace/labels/output/hogwarts/hogwarts_9_features.txt
/home/xi/workspace/labels/output/hogwarts/hogwarts_10_features.txt
/home/xi/workspace/labels/output/hogwarts/hogwarts_11_features.txt
/home/xi/workspace/labels/output/hogwarts/hogwarts_2_features.txt
/home/xi/workspace/labels/output/hogwarts/hogwarts_3_features.txt
/home/xi/workspace/labels/output/hogwarts/hogwarts_4_features.txt
/home/xi/workspace/labels/output/hogwarts/hogwarts_5_features.txt
/home/xi/workspace/labels/output/hogwarts/hogwarts_6_features.txt
/home/xi

In [5]:
def add_data(list):
    x_c = []
    y_c = []
    x_g = []
    y_g = []
    x_v = []
    y_v = []
    for i in list:
        x_c.extend(x_c_list[i-1])
        x_v.extend(x_v_list[i-1])
        x_g.extend(x_g_list[i-1])
        
        y_c.extend(y_c_list[i-1])
        y_v.extend(y_v_list[i-1])
        y_g.extend(y_g_list[i-1])
    return x_c, x_v, x_g, y_c, y_v, y_g

In [6]:
x_c_list = [x_1_c, x_2_c, x_3_c, x_4_c, x_5_c, x_6_c, x_7_c]    
x_v_list = [x_1_v, x_2_v, x_3_v, x_4_v, x_5_v, x_6_v, x_7_v]   
x_g_list = [x_1_g, x_2_g, x_3_g, x_4_g, x_5_g, x_6_g, x_7_g]   

y_c_list = [y_1_c, y_2_c, y_3_c, y_4_c, y_5_c, y_6_c, y_7_c]    
y_v_list = [y_1_v, y_2_v, y_3_v, y_4_v, y_5_v, y_6_v, y_7_v]   
y_g_list = [y_1_g, y_2_g, y_3_g, y_4_g, y_5_g, y_6_g, y_7_g]   


x_train_c, x_train_v, x_train_g, y_train_c, y_train_v, y_train_g = add_data([1, 2])
x_test_c, x_test_v, x_test_g, y_test_c, y_test_v, y_test_g = add_data([3, 4, 5, 6])



In [7]:
names = [
        # "Nearest Neighbors",
        # "Linear SVM", "RBF SVM",
         "Decision Tree", 
        #  "Random Forest", 
        #  "Neural Net", 
        #  "AdaBoost",
        #  "Naive Bayes"
         ]

classifiers = [
    # KNeighborsClassifier(3),
    # SVC(kernel="linear", C=0.025),
    # SVC(gamma=2, C=1),
    DecisionTreeClassifier(max_depth=4),
    # RandomForestClassifier(max_depth=5, n_estimators=15),
    # MLPClassifier(alpha=1),
    # AdaBoostClassifier(),
    # GaussianNB()
    ]

In [ ]:
x_train = x_train_c
y_train = y_train_c
x_test = x_test_c
y_test = y_test_c
print x_train
for name, clf in zip(names, classifiers):
    print name
    clf.fit(x_train, y_train)
    # scores = clf.score(x_test, y_test)
    predict = clf.predict(x_test)
    print metrics.confusion_matrix(y_test, predict)
    print metrics.classification_report(y_test, predict)

#     draw_result_img(x_test_c, y_test_c, x_train_c, y_train_c, img_lengths_test_c, img_uvs_test_c, 'c')
    # draw_result_img(x_test_v, y_test_v, x_train_v, y_train_v, img_lengths_test_v, img_uvs_test_v, 'v')
    # draw_result_img(x_test_g, y_test_g, x_train_g, y_train_g, img_lengths_test_g, img_uvs_test_g, 'g')


In [9]:
############################### roc ############################
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

# fpr_c, tpr_c, roc_auc_c = get_result(x_train_c, x_test_c, y_train_c, y_test_c)
# fpr_v, tpr_v, roc_auc_v = get_result(x_train_v, x_test_v, y_train_v, y_test_v)
fpr_g, tpr_g, roc_auc_g = get_result(x_train_g, x_test_g, y_train_g, y_test_g)

# draw_ROC(fpr_c, tpr_c, roc_auc_c, fpr_v, tpr_v, roc_auc_v, fpr_g, tpr_g, roc_auc_g)

plt.figure(figsize=(5, 5))
#     plt.plot(fpr_c[index], tpr_c[index],
#             label=r'Fusion ({0:0.2f}\% AUC)'.format(100 *roc_auc_c[index]),
#             color='deeppink', linestyle='-', linewidth=2)  
index = 0
plt.plot(fpr_g[index], tpr_g[index],
        label=r'Geometry ({0:0.2f}\% AUC)'.format(100 * roc_auc_g[index]),
        color='deeppink', linestyle='-', linewidth=2)
index = 1
plt.plot(fpr_g[index], tpr_g[index],
        label=r'Geometry ({0:0.2f}\% AUC)'.format(100 * roc_auc_g[index]),
        color='cornflowerblue', linestyle='-', linewidth=2)
index = 2
plt.plot(fpr_g[index], tpr_g[index],
        label=r'Geometry ({0:0.2f}\% AUC)'.format(100 * roc_auc_g[index]),
        color='darkorange', linestyle='-', linewidth=2)
#     plt.plot(fpr_v[index], tpr_v[index],
#             label=r'Vision ({0:0.2f}\% AUC)'.format(100 * roc_auc_v[index]),
#             color='darkorange', linestyle='-', linewidth=2)    
plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel(r'False Positive Rate', fontsize=14)
plt.ylabel(r'True Positive Rate', fontsize=14)
# plt.title('Classification result on ' + type + ' terrain')
plt.legend(loc="lower right")
# plt.savefig('roc_' + type, bbox_inches='tight')
plt.show()



# fpr, tpr, roc_auc = get_result(x_train_c, x_test_c, y_train_c, y_test_c)
# plt.figure()
# plt.plot(fpr["micro"], tpr["micro"],
#         label='Combined ROC curve (area = {0:0.2f})'
#             ''.format(roc_auc["micro"]),
#         color='deeppink', linestyle='-', linewidth=4)

/usr/lib/python2.7/dist-packages/scipy/sparse/compressed.py:130: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  if np.rank(self.data) != 1 or np.rank(self.indices) != 1 or np.rank(self.indptr) != 1:
/usr/lib/python2.7/dist-packages/scipy/sparse/coo.py:200: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  if np.rank(self.data) != 1 or np.rank(self.row) != 1 or np.rank(self.col) != 1:


Decision Tree


/usr/lib/python2.7/dist-packages/scipy/sparse/coo.py:182: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  if np.rank(M) != 2:


ValueError: setting an array element with a sequence.